# Zillow Prize: Zillow’s Home Value Prediction (Zestimate)
## Can you improve the algorithm that changed the world of real estate?
## Josh Wilson, Keane Johnson, Carlos Sancini

## Description
Zillow's "Zestimates" are estimated home values based on numerous categorical and numerical features of a given property. Zillow has been able to improve the median margin of error from 14% at the Zestimate's release to 5% as of 2017. However, they have introduced a competition to Data Scientists to further improve the accuracy of the Zestimate. The goal of this project is to produce a model that  accurately predicts the logerror produced by the Zestimate algorithm's attempt to predict the price of a house. Success is measured by minimizing the mean absolute error between our prediction and the Zestimate log-error. The log-error is defined as:
  
           logerror = log(Zestimate) - log(SalePrice)
           
           MAE = SIGMA |predicted logerror - logerror| / n

We have been provided data on a variety of different features of Los Angeles-area homes sold in the year 2016. Some of these features are numerical and some are categorical. We will be using this data to build our model.

## Exploratory Data Analysis

> ### Import Libraries

In [ ]:
# Packages for manipulating data
import numpy as np 
import pandas as pd
from tabulate import tabulate

# Packages for plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# Stats et al.
import scipy.stats as st
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Model packages
from sklearn import ensemble, tree, linear_model
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.cluster import KMeans

# Support vector machine 
from sklearn.svm import SVR

# Various Tree Based Regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Packages for measuring model performance
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import mean_absolute_error

# general packages
import os
import warnings
warnings.filterwarnings('ignore')

# Setting the seed for reproducibility of results
np.random.seed(27)

> ### Load and Examine the Data

In [ ]:
# read in training and test data and labels

train_data = pd.read_csv('../input/properties_2016.csv')
train_labels = pd.read_csv('../input/train_2016_v2.csv')
#test_data = pd.read_csv('properties_2017.csv')
#test_labels = pd.read_csv('train_2017.csv')

The first step in our exploratory data analysis is to get a basic understanding of our dataset through a quick summary of what it contains.

In [ ]:
train_data.describe()

This summary shows that there are 53 different variables provided in our dataset. The first thing that stands out is the first row of the table returned by the describe() function - *count*. This refers to the number of values in the dataset for that respective column. There are just under 3 million parcelid's, which uniquely identify properties (our observations). However, most of the features of our properties do not have a similar number for *count*. This means that we are missing data for many different features for many different observations. We will take a closer examination of the missing data later on.

The second thing that stands out is that there are numeric values representing categorical features. For example, the column *decktypeid* represents the type of deck of a property. The number assigned to the type of deck does not measure magnitude and could have been arbitrary. 

Finally, we should note that the mean of the majority of the truly numeric (and not representing categorical) features is greater than the median. This means that our features have a positive skew.

Now that we have some general context to our dataset, we can look at the data itself.

In [ ]:
train_labels.head()

The training labels dataset consists of three different columns. *parcelids* identifies individual homes. *logerror* is the difference between the log of Zillow's estimated selling price and the log of the home's actual selling price. And *transactiondate* is the date the home sold.

In [ ]:
train_data.head()

In [ ]:
## Distribution of Target Variable

The below graphs of the target variable reveal three important aspects about the variable we are modeling:
* We are dealing with a relatively normal distribution. Remember that these are essentially results from a model. That means the errors are conditionally normal and we might be able to leverage this fact in our own modeling endeavor.
* The peak around 0 is relatively higher than what we would expect from a normal distribution. 
* Most importantly, the tails of this normal distribution are rather long, meaning we might have to cope with some outliers in our own modeling.

In [ ]:
# Investigating the distribution of the logerrors
#train_labels.logerror.describe())

# plotting the log error graphs side-by-side, one with the full range of values and one with a trimmed range
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
sns.distplot(train_labels['logerror'], bins = 50, kde = False)
plt.title('Histogram of logerror raw')

ulimit = np.percentile(train_labels.logerror.values, 99)
llimit = np.percentile(train_labels.logerror.values, 1)

# creating an array with limited logerror values
logerror = train_labels.logerror.values
logerror = logerror[np.where(logerror < ulimit)]
logerror = logerror[np.where(logerror > llimit)]


plt.subplot(1, 2, 2)
sns.distplot(logerror, bins = 50, kde = False)
plt.title('Histogram of logerror trimmed')



> ### Examine Missing Data

By looking at some example observations of our training data, we can confirm that lots of observations lack complete data. We can further understand the extent of these missing values through a visualization.

In [ ]:
plt.figure(figsize=(20,20))

sns.heatmap(
    train_data.isnull(),
    cbar=False,
    yticklabels=False,
    cmap = 'viridis'
)

This heatmap visualizes whether the training data has missing values. The yellow sections of the chart indicate missing data. This confirms our impression that there is missing data for many of the features. To further understand the extent of missing data, we can calculate the percentage of missing observations by feature.

In [ ]:
total = train_data.isnull().sum().sort_values(ascending=False)
percent = round((train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False), 4)
missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Total Missing Count', '% of Total Observations'])
missing_data.index.name = 'Feature'

missing_data

Unsurprisingly, the variables for which the data are most complete revolve around location, bedroom, bathroom, and tax values. For many people, these are the most important factors in choosing where to live: commute to work/school district, the ability to house a family, and the yearly cost of living in the house. There is then a fairly precipitous dropoff for the next tier of variables and the degree of completeness (e.g. heating type and building type); there does not appear to be any logical explanation, available to us, for why these variable might be more or less complete than others. Lastly, we are missing data for almost all observations for seventeen features. This is unfortunate because some of these variables contain information about potentially impactful features of the value of properties, such as the architecture style, and the size of the basement and pool. Our hope is that these sparse variables can be leveraged in some way to help inform our model.

> ### Correlations Between Dependent and Independent Variables

The next step of our Exploratory Data Analysis is to find correlations between our dependent and independent variables. This will help us identify instances of potential multicollinearity within our independent variables. It will also help us identify features that change with logerror.

In [ ]:
train_data_labels = pd.merge(train_data, train_labels, on='parcelid')

correlation = train_data_labels.corr()

plt.figure(figsize=(20,20))
plt.title('Correlation of Features with logerror',y=1,size=20)

sns.heatmap(
    correlation,
    square=True,
    vmax=0.8
)

We consider the investigation of this correlation matrix heatmap to be a great exercise in building intution around the variables and their relationships to each other. Our discussion of the variables below creates some hypotheses for what we expect a machine learning model to uncover.

It is tough to ignore the very white lines that segment this heatmap. They are the result of sparse data and are not perfectly correlated variables. For example, there is no logical reason that *assessmentyear* and *decktypeid* ought to be correlated. Instead, these are variables with low mass.

Lighter squares on this heat map indicate that the variables are correlated. There are five squares that stick out. The first is between *calculatedfinishedsquarefeet* and *finishedsquarefeet12*. Both of these variables actually capture the same features of a home - the finished living area. 

Other lighter squares are *structuretaxvaluedollarcnt*, which measures the value of the structures on the property, *taxvaluedollarcnt*, which measures the value of the property, *landtaxvaluedollarcnt*, which measures the value of the land on the property, and *taxamount*, which is the total tax assessed. It is unsurprising that these variables are highly related to each other. Regardless of their similarity, Zillow decided to include *all of them* in the data provided to contestants. There are a couple of reasons why these variables might prove to be useful. First, the completeness of data collection for each variable might differ, so there might be value in the presence of the variable. And second, they measure slightly different things and variation across different tax areas in L.A. might be meaningful for the purchase of a home.

There are some surprisingly strong connections between variables one might not have thought would display such strong correlations. For example,  the *regioncountyid* and *airconditioningtypeid* display a very strong negative correlation. One could imagine that newer houses might be equipped with central air conditioning while older houses may have window units. However, the connection to region county seems tenuous unless there is some regulation or code in place that requires certain types of air condition vs. others.

Another interesting strongly negative correlation arises between the *propertylandusetypeid* variable and *garagetotalsqft*. We wouldn't expect an apartment to have a very large garage (or maybe we should depending on how this variable is measured) while a mansion in Bel Air could have a garage the size of some other homes. Lack of transparency on variable measurement degrades trust in the modeling exercise.

Finally, we can visualize how a sample of these variables are correlated with *logerror* through pair plots. We are choosing to visualize features that are present for more than 90% of observations and are not some sort of numeric identifier. For example, although most properties contain *censustractandblock*, we are not including it in this visualization because it is census ID. This 90% cutoff is being used for this visualization but will not necessarily be adhered to for our model-building.

What we care most about in this display are the bivariate plots with logerror.  Remember from the above graph of the logerror that there is a strong presence of outliers. We can use the below graph to see if the logerror outliers are associated with any variables in particular. There are several bivariate plots that have a reverse funnel shape, meaning that the outliers for logerror are located at the bottom and that prediction for the houses with features higher up the y - axis (if you look in the first column). Logerror's relationships with *roomcnt* is particularly interesting, because it seems to suggest that Zillow's algorithm does not make accurate predictions for housing with lower *roomcnt*, and there is a decent spread as the *roomcnt* grows until the accuracy increases rather drastically. In many other cases, the larger the house or the more amount of tax dollars associated with the house, the more accurate prediction. Perhaps there are some mediating variables that make the prediction better for these houses, such as recency of the previous purchase or perhaps recent construction. 

In [ ]:
sns.set()

columns = [
    'logerror', 'lotsizesquarefeet', 'finishedsquarefeet12','landtaxvaluedollarcnt', 
    'calculatedfinishedsquarefeet', 'taxamount', 'roomcnt', 'bathroomcnt', 'bedroomcnt'
]

sns.pairplot(
    train_data_labels[columns], 
    size=2, 
    kind ='scatter', 
    diag_kind='kde'
)

## Preprocessing and feature engineering

As shown before, our dataset has several features that have a high percentage of missing values. To create our model, it will be considered only features that have less than 10% missing values. Exceptions to this rule were applied when  based on descriptions of the data dictionary and filled values:

- *poolcnt*: the data dictionary provided by Kaggle mentions that this feature related to the number of pools on the lot (if any). This feature will also be combined with other pool related features to create derived variable that indicates the presence of a pool. Missing values will be filled with zeros (absence of a pool).

- *taxdelinquencyflag*: the data dictionary mentions "Property taxes for this parcel are past due as of 2015" and there are only 'Y' values filled. Missing values will be filled with zeros ('N' not delinquent).

- *taxdelinquencyyear*: the data dictionary mentions "year for which the unpaid property taxes were due"  and only 'Y'. The feature will be transformed to the total number of years of delinquency. Missing values will be filled with zeros (not delinquent).

- *buildingqualitytypeid*: the data dictionary mentions "overall assessment of condition of the building from best (lowest) to worst (highest)" and, although this feature has 35% of NAs, its an important feature and missing values will be filled with mean values.

- *heatingorsystemtypeid*: the data dictionary mentions "type of home heating system" and 39% of values are missing. An "other" value will be considered for this feature as 95% of households in US have a heating system (According to the 2015 report from U.S. Energy Information Administration)

The mean value assignments for missing values will be done later on based on a clustering strategy, i.e., the mean value considered will be the one of the cluster that the observation belongs to. 

In [ ]:
# pool treatment
train_data['has_pool'] = ((train_data.poolcnt > 0) | (train_data.poolsizesum > 0) | (train_data.pooltypeid10 == 1) | 
                          (train_data.pooltypeid2 == 1) | (train_data.pooltypeid7 == 1)).astype(int)

# tax delinquency treatment
train_data['is_delinquent'] = (train_data.taxdelinquencyflag == 'Y').astype(int)
def conditions(x): 
    v = 0
    if np.isnan(x): 
        return 0
    if x > 15:
        return int(2016 - (x + 1900))
    else:
        return int(2016 - (x + 2000))
train_data['years_of_delinquency'] = train_data['taxdelinquencyyear'].apply(conditions)

### features with less 10% NAs

# characteristics of property
cols_property_characteristics = [
    'roomcnt', 
    'bedroomcnt',
    'bathroomcnt',
    'fullbathcnt',
    'calculatedbathnbr',
    'calculatedfinishedsquarefeet',
    'finishedsquarefeet12',
    'lotsizesquarefeet',
    'yearbuilt',
    'assessmentyear',
    'has_pool'] 

# tax information about property (proxy for value)
cols_tax_info = ['taxamount', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt',
                 'landtaxvaluedollarcnt', 'is_delinquent', 'years_of_delinquency']

# use of property (categorical)
cols_property_use = ['propertylandusetypeid', 'propertycountylandusecode']

# locality (categorical)
cols_locality = ['regionidcounty', 'regionidcity', 'regionidzip'] 
# other locality (no NAs but possibly redundants with locality) 
# cols_other_locality = ['censustractandblock', 'rawcensustractandblock', 'fips', 'longitude', 'latitude']

# sets a value for NAs
train_data[cols_property_use + cols_locality] = train_data[cols_property_use + cols_locality].fillna('unknown')

# drop NA values
selected_features = cols_locality + cols_property_characteristics + cols_property_use + cols_tax_info
train_data_no_NAs = train_data[selected_features].dropna()

print("Original dataset:", train_data.shape)
print("Dataset without NAs:", train_data_no_NAs.shape)

A data standardization is applied to selected features to prepare them for clustering. Leaving variances unequal will lead the KMeans algorithm to put more weight on variables with smaller variance. The features selected for clustering are related to property characteristics and value (proxied by tax value). Categorical variables were dropped from the analysis due to potential inconsistencies that these variables could cause (Euclidean distances usually do not make sense for categorical data). Besides filling NAs values, the clusters will also be used as new features for the modeling step as they could encode non-linarities that the linear regression would not identify. The clusters will also be useful to analize the centroid values and understand the housing market.  

# standardizes features
df_to_scale = train_data_no_NAs[cols_property_characteristics + cols_tax_info]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_to_scale.values)
scaled_data = pd.DataFrame(scaled_data, columns=df_to_scale.columns, index=df_to_scale.index)
scaled_data.shape

scaled_data[sorted(scaled_data.columns)].head()

To identify the optimal *K*, the Elbow method is used. The Kmeans clustering is executed multiple times with different values of k and with a smaller number of observations, which were sampled from original dataset. This choice was made due to performance issues. The result is show in the plot below.   

# run clustering with less data to find optimal k
clustering_data = scaled_data.sample(n=50000)

distortions = []
K = range(1,50)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++', n_jobs=-1).fit(clustering_data)
    distortions.append(sum(np.min(cdist(clustering_data, km.cluster_centers_, 'euclidean'), axis=1)) / clustering_data.shape[0])

# Plot the elbow
plt.figure(figsize=(10,10))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow method showing the optimal k')
plt.show()

By analysing the plot, it can be seems that the elbow is achieved when k is 13. The algorithm is executed once more with all data and the optimal k. 

# Runs the clustering with k=13 based on elbow analysis
optimal_k = 13
km = KMeans(n_clusters=optimal_k, init='k-means++', n_jobs=-1).fit(scaled_data)

# scale back and presents typical values for each cluster
centroids = scaler.inverse_transform(km.cluster_centers_)
centroids = pd.DataFrame(centroids, columns=scaled_data.columns)
centroids.round(0)

An iterative EM-type algorithm will be used to fill NAs based on the following steps: 

1) Initialize missing values to their column means

2) Repeat until convergence:

- Perform K-means clustering on the filled-in data

- Set the missing values to the centroid coordinates of the clusters to which they were assigned

By this method we will achieve aproximate mean values for missing data. The mean value for each row/feature will not be the mean value for the entire dataset then, but the mean value of its corresponding cluster.

def kmeans_missing(X, n_clusters, max_iter=20):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      initial_centroids
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)
    
    # Standardizes features
    scaler = StandardScaler()
    X_hat = scaler.fit_transform(X_hat)
    
    for i in range(max_iter):
        
        if (i % 10) == 0:
            print("Iteration:", i)
        
        if i > 0:
            # initialize KMeans with the previous set of centroids. 
            cls = KMeans(n_clusters, init=prev_centroids, n_jobs=-1)
        else:
            cls = KMeans(n_clusters, n_jobs=-1)

        # perform clustering on the filled-in data
        predicted_labels = cls.fit_predict(X_hat)
        predicted_centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = predicted_centroids[predicted_labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(predicted_labels == prev_labels):
            print("Clusters converged after", i, "iterations")
            break

        prev_labels = predicted_labels
        prev_centroids = predicted_centroids 

    # Scale back values
    new_centroids = pd.DataFrame(scaler.inverse_transform(predicted_centroids), columns=X.columns)
    X_hat = pd.DataFrame(scaler.inverse_transform(X_hat), columns=X.columns, index=X.index)
    
    return predicted_labels, new_centroids, X_hat, missing, cls, scaler

# obtain dataset with NAs filled based on clusters means
labels, new_centroids, train_data_no_NA, missing, cls, scaler = kmeans_missing(
    train_data[cols_property_characteristics + cols_tax_info], optimal_k)

The table below shows the new clusters obtained after several iterations.

new_centroids.round(0)

The obtained clusters are added as a new feature to the dataset, all numerical types are converted to integers and all categorical type are converted to panda's category data type.

# adding clusters as a variable
train_data_no_NA['cluster'] = labels

# converting data types and joining categorical and non catgorical data in the final dataset
prepared_data = pd.concat(
    [train_data_no_NA.round(0).astype('int64'), train_data[cols_locality + cols_property_use]], 
    axis=1, 
    sort=False)
cols = cols_locality + cols_property_use + ['cluster', 'is_delinquent', 'has_pool']
prepared_data[cols] = prepared_data[cols].astype('category')

print(prepared_data.shape)
print(prepared_data.dtypes)

pd.set_option('display.max_columns', 25)
prepared_data[sorted(prepared_data.columns)].head()

The prepared dataset, the kmeans model and the scaler model are then persisted. 

# persists the prepared dataset
prepared_data.to_pickle('./prepared_data.pkl')

# load the persisted dataset 
# prepared_data = pd.read_pickle("./prepared_data.pkl")

# persists the kmeans model
joblib.dump(cls, './kmeans_model.joblib') 

# load the kmeans  model
# cls = joblib.load('kmeans_model.joblib') 

# persists the scaler
joblib.dump(scaler, './scaler.joblib') 

# load the scaler
# scaler = joblib.load('scaler.joblib') 

## Training Data Set for Baseline Model

For training the baseline model, we ignore any of the feature engineering that we've done previously and any missing value imputation. In this first pass, we want to use all of the variables to see what pops as important and what doesn't. We use a simple measure to fill in the missing data with 0s knowing that imputed values potentially offer us some low hanging fruit for improving the model.

In [ ]:
# transforming data type in place
for c, dtype in zip(train_data.columns, train_data.dtypes):
    if dtype == np.float64:
        train_data[c] = train_data[c].astype(np.float32)
  
# First we are going to make sure the training data only has attributes for the training labels
train_df = train_data.merge(train_labels, how = 'inner', left_on = 'parcelid', right_on = 'parcelid')
print(train_df.shape)
y = train_df['logerror']

# creating train and validation sets
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2)

# remove unnecessary columns from train_df
X_train = X_train.drop(['parcelid', 'logerror','transactiondate', 'propertyzoningdesc',
                  'taxdelinquencyflag', 'propertycountylandusecode','hashottuborspa','fireplaceflag'], axis=1)
X_test = X_test.drop(['parcelid', 'logerror','transactiondate', 'propertyzoningdesc',
                  'taxdelinquencyflag', 'propertycountylandusecode','hashottuborspa', 'fireplaceflag'], axis=1)

# filling in missing values with a zero...stop gap solution
X_train.fillna(0, inplace= True)
X_test.fillna(0, inplace= True)

## Baseline Models

To start, we use 5 different machine learning models: Lasso, Random Forest, Decision Trees, Xtreme Gradient Boosted Machine, and Light GBM. The hope is that these models will vary in performance, suggesting a model that we could potentially grid search and refine for the best performance. We will also be able to investigate how the different tree models weight the importance of the variables, which we can use to inform further feature creation and possibly model selection. Lastly, we can look to see where these baseline models perform worse to understand if there is any way to improve performance.

In [ ]:
# storing model object with basic parameters, we didn't standardize prior, so we normalize here
# alpha set to near 0 is essentially a linear regression without penalties
ls = Lasso(alpha=1e-6, normalize=True)

# Various Tree Models
rf = RandomForestRegressor(n_estimators = 12, max_depth = 5)
dt = DecisionTreeRegressor(max_depth = 4)
xg = XGBRegressor(n_jobs = 1)
lg = lgb.LGBMRegressor(num_leaves = 31,
                        learning_rate = 0.05,
                        n_estimators = 20)

# Other models that were compared
#sv = SVR(kernel='linear', C=1e3)
#ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),n_estimators=300, random_state=seed)
#cb = CatBoostRegressor(iterations=300, learning_rate=0.01,
#                        depth=4, l2_leaf_reg=3,
#                        loss_function='MAE',
#                        eval_metric='MAE',
#                        random_seed=seed)

## Feeding a bunch of baseline model objects with set parameters into a function and have it return
## various details

# Storing model names and their associated objects
model_names = ['Lasso','DecisionTrees', 'Random Forest', 'Xtreme Gradient Boosted', 'Light GBM']
model_objects = [ls, dt, rf, xg, lg]

# Creating a dictionary for storing different model results
model_output = {"model_names": model_names, "model_objects": model_objects, "mean_abs_error": [], 
                "predictions": [], "variable_importance": []}

# Creating empty lists for storing model outputs for the dictionary
mae = []
variable_importance = []
predictions = []

# Function to run through models 
def run_models(model_objects, X_train, y_train, X_test, y_test):
    for model in model_objects:
        model.fit(X_train, y_train)
        p = model.predict(X_test)
        mae.append(mean_absolute_error(y_test, p))
        predictions.append(p)
        try:
            variable_importance.append(model.feature_importances_)
        except: 
            variable_importance.append("no such attribute")
    model_output["mean_abs_error"] = mae
    model_output["predictions"] = predictions
    model_output["variable_importance"] = variable_importance
    return(model_output)

# fit and predict all of the models using the function above
run_models(model_objects, X_train, y_train, X_test, y_test)

In [ ]:
# A Nicer Table of the Baseline Models
from tabulate import tabulate
['Lasso', 'DecisionTrees', 'Random Forest', 'Xtreme Gradient Boosted', 'Light GBM']
print(tabulate([['Lasso', model_output["mean_abs_error"][0]], 
                ['DecisionTrees', model_output["mean_abs_error"][1]],
                ['Random Forest', model_output["mean_abs_error"][2]],
                ['Xtreme Gradient Boosted', model_output["mean_abs_error"][3]],
                ['Light GBM', model_output["mean_abs_error"][4]]
               ], headers=['Model Name', 'MAE'], tablefmt='orgtbl'))

All of the models chosen are in within very close proximity with regards to performance. The Light GBM has shown the best performance of all of the models and we will be moving forward with it for our grid search.

## Feature Importance for Models

In [ ]:
# plotting feature importances
def plot_importances(model_name, i, model_output, top_n):
    # we only want to plot the variable importance for those models with that attribute
    if model_output["variable_importance"][i] != "no such attribute":
        # extract from model output dictionary and store as a dataframe
        feat_imp = pd.DataFrame({'importance': model_output["variable_importance"][i]})    
        
        # grab the column names then sort in place
        feat_imp['feature'] = X_train.columns
        feat_imp.sort_values(by='importance', ascending=False, inplace=True)
        
        # grabbing only the top n features according to their weight
        feat_imp = feat_imp.iloc[:top_n]
        feat_imp.sort_values(by='importance', inplace=True)
        feat_imp = feat_imp.set_index('feature', drop=True)
        
        # plotting components
        feat_imp.plot.barh(title="Variable Importance {}".format(model_name), figsize=(10,10))
        plt.xlabel('Feature Importance Score')
        plt.show()
    else: print("no variable importance attribute for {}".format(model_name))

# loop through models and plot the various variable importances
j = 0
for model_name in model_names:
    plot_importances(model_name, j, model_output, top_n = 15)
    j = j + 1

There are remarkable differences on variable importance across the models. Despite having the fewest variables in the graph, decision trees still perform remarkable well compared to the other models. The fact that latitude matters a lot across the different models comes as no surprise. The farther West one goes in L.A. the closer to the water one gets. That is likely to come at a premium and there is likely to be some turnover at these kinds of locations. Random forest and Xtreme gradient boosting both appear to prioritize tax money and size of the building before getting to variables regarding location. While light gbm contains a different set of variables, placing a lot of importance on the zip code and finished square feet. Bedroom and bathroom counts, something that most of us consider to be very important when buying a house (I would know, I just bought one this month!), rarely show up and show up with no consistency.

## Correlation between model predictions

In [ ]:
# Determining if the model predictions are highly related across models

# create a numpy array that contains all of the prediction for each model as column vectors
merged_array = model_output['predictions'][0]

k = 1
for k in range(1, len(model_names)):
 merged_array = np.vstack((merged_array, model_output['predictions'][k]))
merged_array = merged_array.T
dataset = pd.DataFrame({'Lasso':merged_array[:,0],'DecisionTrees':merged_array[:,1], 
                        'Random Forest':merged_array[:,2], 'Xtreme Gradient Boosted':merged_array[:,3], 
                        'Light GBM': merged_array[:,4]})

# Calculate the correlation matrix
corr = dataset.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

We just witnessed the variability in the variables used to construct trees. We were curious how related the model predictions would be correlated across the baseline models. It turns out that they are not very correlated. This suggests that it might be a worthwhile exercise to combine these models via an ensemble method. That's beyond the scope of this notebook but definitely a worthy avenue to pursue.

## Error Analysis

In [ ]:
# finding the largest errors made by a model
i = 0
def find_errors(model_output, y_test, i, number):
    
    # extract the predictions from the dictionary, note that this is a series object
    distances = abs(model_output["predictions"][i] - y_test)
    distances_srt = distances.sort_values(ascending = False)
    # extract top 100 largest values and return their indices
    return(distances_srt.keys()[range(0,number)])

# Choose model and inputs
i = 2
errors = find_errors(model_output, y_test,i,50)

# Let's take a look at the target values
y_test[y_test.index.isin(errors)]



In [ ]:
X_test[X_test.index.isin(errors)]

A couple of items are relatively obvious after looking at the data for which our baseline models have made the largest errors. First, we have a problem being able to accurately predict the outliers in the data, as seen by the very large test values. Second, there is no obvious pattern for why these predictions are being missed. Property land use, region, and tax amounts vary across the largest errors. The one variable that does stand out on investigation is the yearbuilt. It looks as though most of the error for the random forest model are for those houses built before 2000. Perhaps, a binary variable for the year built will help with the outliers.

In [ ]:
def plot_error_location(errors, X_test):
    plot_df = X_test[X_test.index.isin(errors)]
    plt.figure(figsize = (12,12))
    sns.jointplot(x = plot_df.latitude.values, y = plot_df.longitude.values, size=10)
    plt.ylabel('Longitude', fontsize =  12)
    plt.xlabel('Latitude', fontsize = 12)
    plt.show()
    
plot_error_location(errors, X_test)

The above graph gives some perspective on the location of our largest errors. A more rigorous feature set might contain new ways of dividing up the map. It seems that the no location variable does a sufficiently good job on its own.

## Grid Search of LGM

We've chosen to grid search the LGM model based on the preliminarily better results that were achieved above. The LGM model also performs well with regards to speed, which matters in this case because the grid search is running on a relatively small compute node.  For example, a relatively small grid search, such as the one below, could take nearly 30 minutes to run. 

In the grid search below and other grid searches that we ran, we varied the learning rate, estimators, and number of leaves. Grid searching the learning rate hyperparameter gives us the best shot at finding a true minimum, hopefully without getting stuck at a local minimum. The number of leaves will control the complexity of the tree, with more leaves representing a more complex tree. And lastly the 

Generally, we found that the smaller learning rate increases our predictive ability, but only marginally and at the cost of the training taking more time. Accuracy also increased with the number of leaves until the complexity became too great and we started overfitting the model. This is when we started including some of the other hyperparameters that control resampling. These were implemented to combat overfitting as we increased the complexity of the model.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create parameters to search
param_grid = {
    'learning_rate': [0.01, 0.1], # size of the steps we take on the search
    'n_estimators': [20,40,60],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'], # default setting for gradient boosted trees
    'objective' : ['regression_l1'], # this is the equivalent to mean absolute error
    'colsample_bytree' : [0.65, 0.66], # helps us with overfitting
    'subsample' : [0.7,0.75],# how we resample from the data
    'lambda_l1' : [1,1.2], # regularization parameter
    'lambda_l2' : [1,1.2,1.4],# regularization parameter
    }

# Create regressor to use. with default parameters as input
mdl = lgb.LGBMRegressor(num_leaves = 31)

# Create the grid
grid = GridSearchCV(mdl, param_grid,
                    verbose=0,
                    cv=4,
                    n_jobs=2)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
# Setting parameters based on grid search...don't want to keep running as it takes 20-30 minutes
best_fit = lgb.LGBMRegressor(objective = 'regression_l1',
          n_estimators = 60,
          learning_rate = 0.1,
          subsample = 0.7,
          colsample_bytree = 0.66,
          lambda_l1 = 1,
          lambda_l2 = 1,
          num_leaves = 40)

# fitting the model and seeing how we do on the eval set
best_fit.fit(X_train,y_train,
            eval_set = [(X_test,y_test)],
            eval_metric = 'l1',
            early_stopping_rounds = 5)

## Variable Importance and Error Analysis

Using the same methods created above, we can investigate the importance certain variables in the model. Again, we see that the size of the house and the tax amounts are the most important variables, but not the next set of variables are very different. When the house was built and where it was built end up being very influential. This is logical, because older houses tend to need more work done in order to make them livable. The end result being that the price of the house will decrease with the amount of expected work that is needed for updates. And second, locaition, and specifically lat and lon are a very precise way to determine where some house is on the map. Proxies like zip code and region potentially group together houses that are vastly different.

In [ ]:
# Looking at variable importance
model_output_bf = {"model_names": ['lgb_bestfit'], "model_objects": best_fit,
                   "mean_abs_error": mean_absolute_error(y_test, best_fit.predict(X_test)), 
                "predictions": best_fit.predict(X_test), 
                   "variable_importance": [best_fit.feature_importances_]}
j = 0
plot_importances('lgb_bestfit', j, model_output_bf, top_n = 15)

Unfortunately, we see that the most common errors are still being made on outliers. It does not appear that tuning the model helps us in becoming better predictors of outliers.

In [ ]:
# using the function created above
i = 0
errors = find_errors(model_output_bf, y_test,i,50)

# Let's take a look at the target values
y_test[y_test.index.isin(errors)]

In [ ]:
X_test[X_test.index.isin(errors)]

## Next Steps

We believe there are two fruitful avenues we could pursue to make the model more accurate. First, we might want to take the outliers out of the training set and determine if there are any improvements in our accuracy. Second, we could look for a model that does a better job with predicting the outliers and try a stacking approach in which we combine model predictions to make a better prediction overall.